In [1]:
import warnings 
warnings.filterwarnings("ignore") 
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train_d_beneficiary = pd.read_csv('health_cs_data/Train_Beneficiarydata-1542865627584.csv')
train_d_inpatient = pd.read_csv('health_cs_data/Train_Inpatientdata-1542865627584.csv')
train_d_outpatient = pd.read_csv('health_cs_data/Train_Outpatientdata-1542865627584.csv')
label_d_data = pd.read_csv('health_cs_data/Train-1542865627584.csv')


print("train_data_beneficiary : ",train_d_beneficiary.shape)
print("train_data_inpatient : ",train_d_inpatient.shape)
print("train_data_outpatient : ",train_d_outpatient.shape)
print("label_data : ",label_d_data.shape)

train_data_beneficiary :  (138556, 25)
train_data_inpatient :  (40474, 30)
train_data_outpatient :  (517737, 27)
label_data :  (5410, 2)


Feature Engineering

Now a major factor is determining which features I include is the percentage of Null values present.
-> A feature 'is_beneficiary_present' is not required as %age null values are 0 meaning it is always present.
-> A feature 'is_operating_physician_present' makes sense as there are 79% null values here. 

1. Feature 'whether_admitted'::
   For inpatients the value will be 1 and for outpatients it will be 0 

In [3]:
train_d_inpatient['whether_admitted'] = 1
train_d_outpatient['whether_admitted'] = 0

In [11]:
patient_data = pd.merge(train_d_inpatient, train_d_outpatient, left_on = [ col for col in train_d_outpatient.columns if col in train_d_inpatient.columns], \
                    right_on = [ col for col in train_d_outpatient.columns if col in train_d_inpatient.columns], how = 'outer')

In [12]:
patient_data = pd.merge(patient_data,train_d_beneficiary,how='inner',on='BeneID' ).\
              merge(label_d_data,how='outer',on='Provider')

%age null values in date_of_death column are 99.25%, so I'll include 'is_dead' as a feature
-> wherever 'nan' is present,it may mean that either the person is alive or the data is missing.I'll replace a non null value with 1 meaning the person is dead. Else with 0.

I'll include 'is_alive' as a feature based on the fact that if dod column has non null values,means he is not alive.(equal to 0) else 1.

2. Features 'is_dead' and 'is_alive'::
   For patients with Date_of_Death as NaN,it is 0, else 1 

In [13]:
patient_data.loc[patient_data.DOD.notna(),'is_dead'] = 1
patient_data['is_dead']= 0

In [36]:
patient_data.loc[patient_data.DOD.notna(),'is_alive'] = 0
patient_data['is_alive']= 1

Now I'll add some features which showed a clearer picture is capturing whether a claim is fraud or not in my EDA analysis

3. Feature 'attendphysician_count'
       It implies,the number of times a physician attended a patient.In my EDA analysis, it was evident that if the count is more then it  might be a potential fraud

In [14]:
att_physician_count = patient_data['AttendingPhysician'].value_counts().to_dict()
patient_data['attend_physician_count']=patient_data['AttendingPhysician'].map(att_physician_count)

4. Feature 'operate_physician_count'
It implies,the number of times a physician operated.In my EDA analysis, it was evident that if the count is more then it might be a potential fraud

In [15]:
oper_physician_count = patient_data['OperatingPhysician'].value_counts().to_dict()
patient_data['operate_physician_count']=patient_data['OperatingPhysician'].map(oper_physician_count)

5. Feature 'BeneID_count'
       It implies,the number of times a beneficiary submitted claims.In my EDA analysis, it was evident that if the count is more then it might be a potential fraud. Though a clear demarcation as in physician_count was not obtained here in my EDA.

In [16]:
ben_count = patient_data['BeneID'].value_counts().to_dict()
patient_data['BeneID_count']=patient_data['BeneID'].map(ben_count)

6. Feature 'provider_count'
       It implies,the number of times a provider gave services.In my EDA analysis, it was evident that if the count is more then it might be a potential fraud. And a clear demarcation was obtained here in my EDA which shows it is a useful feature

In [17]:
prov_count = patient_data['Provider'].value_counts().to_dict()
patient_data['provider_count']=patient_data['Provider'].map(prov_count)

7. Feature 'Claim_days'
       It implies,the number of days it took for a claim to be reimbursed.In my EDA analysis, it was evident that if the days are more then it might be a potential fraud,though it was not necessary.

In [18]:
patient_data['Claim_Start'] = pd.to_datetime(patient_data['ClaimStartDt'] , format = '%Y-%m-%d')
patient_data['Claim_End'] = pd.to_datetime(patient_data['ClaimEndDt'],format = '%Y-%m-%d')

patient_data['DOB'] = pd.to_datetime(patient_data['DOB'] , format = '%Y-%m-%d')
patient_data['DOD'] = pd.to_datetime(patient_data['DOD'],format = '%Y-%m-%d')

In [19]:
patient_data['Claim_Days'] = ((patient_data['Claim_End'] - patient_data['Claim_Start']).dt.days) + 1

8. Feature 'hospitalization_days'
       It implies,the number of days patient was admitted.In my EDA analysis, it was evident that if the days are more then it is a potential fraud.

In [20]:
patient_data['Admission_Date'] = pd.to_datetime(patient_data['AdmissionDt'] , format = '%Y-%m-%d')
patient_data['Discharge_Date'] = pd.to_datetime(patient_data['DischargeDt'],format = '%Y-%m-%d')
patient_data['hospitalization_days'] = ((patient_data['Discharge_Date'] - patient_data['Admission_Date']).dt.days) + 1

9. Feature 'total_diff_amount'
       Firstly i'll sum the i/p and o/p reimbursement amounts, then i/p and o/p deductible amounts. Finally the total_diff_amount will be the difference of the two.
       

In [21]:
reimb_amount = patient_data['IPAnnualReimbursementAmt'] + patient_data['OPAnnualReimbursementAmt']
deduct_amount = patient_data['IPAnnualDeductibleAmt'] + patient_data['OPAnnualDeductibleAmt']
patient_data['total_diff_amount'] = reimb_amount - deduct_amount

10. Now I'll include the top 7 diagnosis codes and 7 procedure codes as my 14 new features with a 0/1 value. 

In [22]:
diagnosis_codes = patient_data[['ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10']]

procedure_codes = patient_data[['ClmProcedureCode_1','ClmProcedureCode_2','ClmProcedureCode_3','ClmProcedureCode_4','ClmProcedureCode_5','ClmProcedureCode_6']]

In [23]:
Seven_diag_codes = ['4019','25000','2724','V5869','4011','42731','V5861']  # from EDA    

In [24]:
patient_df = pd.DataFrame(columns = ['procedure'])
patient_df['procedure'] = pd.concat([patient_data["ClmProcedureCode_1"],patient_data["ClmProcedureCode_2"],patient_data["ClmProcedureCode_3"],patient_data["ClmProcedureCode_4"],patient_data["ClmProcedureCode_5"],patient_data["ClmProcedureCode_6"]],axis=0)
patient_df = patient_df.dropna()

patient_df['procedure'].value_counts().head(10)

4019.0    1959
9904.0    1152
2724.0    1054
8154.0    1022
66.0       901
3893.0     854
3995.0     809
4516.0     651
3722.0     589
8151.0     463
Name: procedure, dtype: int64

In [25]:
Seven_proced_codes = ['4019.0','9904.0','2724.0','8154.0','66.0','3893.0','3995.0']

In [26]:
for i in Seven_proced_codes:
    patient_data['Proc_Code_'+str(i)] = np.where(patient_data['ClmProcedureCode_1']==float(i),1,0)
    patient_data['Proc_Code_'+str(i)] = np.where(patient_data['ClmProcedureCode_2']==float(i),1,np.where(patient_data['Proc_Code_'+str(i)]==1,1,0 ))
    patient_data['Proc_Code_'+str(i)] = np.where(patient_data['ClmProcedureCode_3']==float(i),1,np.where(patient_data['Proc_Code_'+str(i)]==1,1,0 ))
    patient_data['Proc_Code_'+str(i)] = np.where(patient_data['ClmProcedureCode_4']==float(i),1,np.where(patient_data['Proc_Code_'+str(i)]==1,1,0 ))
    patient_data['Proc_Code_'+str(i)] = np.where(patient_data['ClmProcedureCode_5']==float(i),1,np.where(patient_data['Proc_Code_'+str(i)]==1,1,0 ))
    patient_data['Proc_Code_'+str(i)] = np.where(patient_data['ClmProcedureCode_6']==float(i),1,np.where(patient_data['Proc_Code_'+str(i)]==1,1,0 ))
    

In [27]:
for i in Seven_diag_codes:
    patient_data['Diag_Code_'+str(i)] = np.where(patient_data['ClmDiagnosisCode_1']==i,1,0)
    patient_data['Diag_Code_'+str(i)] = np.where(patient_data['ClmDiagnosisCode_2']==i,1,np.where(patient_data['Diag_Code_'+str(i)]==1,1,0 ))
    patient_data['Diag_Code_'+str(i)] = np.where(patient_data['ClmDiagnosisCode_3']==i,1,np.where(patient_data['Diag_Code_'+str(i)]==1,1,0 ))
    patient_data['Diag_Code_'+str(i)] = np.where(patient_data['ClmDiagnosisCode_4']==i,1,np.where(patient_data['Diag_Code_'+str(i)]==1,1,0 ))
    patient_data['Diag_Code_'+str(i)] = np.where(patient_data['ClmDiagnosisCode_5']==i,1,np.where(patient_data['Diag_Code_'+str(i)]==1,1,0 ))
    patient_data['Diag_Code_'+str(i)] = np.where(patient_data['ClmDiagnosisCode_6']==i,1,np.where(patient_data['Diag_Code_'+str(i)]==1,1,0 ))
    patient_data['Diag_Code_'+str(i)] = np.where(patient_data['ClmDiagnosisCode_7']==i,1,np.where(patient_data['Diag_Code_'+str(i)]==1,1,0 ))
    patient_data['Diag_Code_'+str(i)] = np.where(patient_data['ClmDiagnosisCode_8']==i,1,np.where(patient_data['Diag_Code_'+str(i)]==1,1,0 ))
    patient_data['Diag_Code_'+str(i)] = np.where(patient_data['ClmDiagnosisCode_9']==i,1,np.where(patient_data['Diag_Code_'+str(i)]==1,1,0 ))
    patient_data['Diag_Code_'+str(i)] = np.where(patient_data['ClmDiagnosisCode_10']==i,1,np.where(patient_data['Diag_Code_'+str(i)]==1,1,0 ))
    
    

10. I'll add 3 new features: is_primary,is_secondary,is_tertiary

In [28]:
patient_data['is_primary'] = np.where(patient_data['AttendingPhysician'].notnull(),1,0)
patient_data['is_secondary'] = np.where(patient_data['OperatingPhysician'].notnull(),1,0)
patient_data['is_tertiary'] = np.where(patient_data['OtherPhysician'].notnull(),1,0)

11. I'll do some imputation related stuff now

1) Replace the yes/no in PotentialFraud column with 1/0

In [23]:
patient_data['PotentialFraud'].value_counts()

No     345415
Yes    212796
Name: PotentialFraud, dtype: int64

In [34]:
patient_data['PotentialFraud'] = np.where(patient_data['PotentialFraud']=='Yes',1,0)

In [35]:
patient_data['PotentialFraud'].value_counts()

0    345415
1    212796
Name: PotentialFraud, dtype: int64

2) Replace the y/0 in RenalDiseaseIndicator with 1/0

In [26]:
patient_data['RenalDiseaseIndicator'].value_counts()

0    448363
Y    109848
Name: RenalDiseaseIndicator, dtype: int64

In [31]:
patient_data['RenalDiseaseIndicator'] = np.where(patient_data['RenalDiseaseIndicator']=='Y',1,0)

In [32]:
patient_data['RenalDiseaseIndicator'].value_counts()

0    448363
1    109848
Name: RenalDiseaseIndicator, dtype: int64

12. Adding 6 new features which indicate the count of top 3 diagnosis and procedure codes

In [29]:
ClmProcedureCode_1_count = patient_data['ClmProcedureCode_1'].value_counts().to_dict()
patient_data['ClmProcedureCode_1_count']=patient_data['ClmProcedureCode_1'].map(ClmProcedureCode_1_count)

ClmProcedureCode_2_count = patient_data['ClmProcedureCode_2'].value_counts().to_dict()
patient_data['ClmProcedureCode_2_count']=patient_data['ClmProcedureCode_2'].map(ClmProcedureCode_2_count)

ClmProcedureCode_3_count = patient_data['ClmProcedureCode_3'].value_counts().to_dict()
patient_data['ClmProcedureCode_3_count']=patient_data['ClmProcedureCode_3'].map(ClmProcedureCode_3_count)

ClmDiagnosisCode_1_count = patient_data['ClmDiagnosisCode_1'].value_counts().to_dict()
patient_data['ClmDiagnosisCode_1_count']=patient_data['ClmDiagnosisCode_1'].map(ClmDiagnosisCode_1_count)

ClmDiagnosisCode_2_count = patient_data['ClmDiagnosisCode_2'].value_counts().to_dict()
patient_data['ClmDiagnosisCode_2_count']=patient_data['ClmDiagnosisCode_2'].map(ClmDiagnosisCode_2_count)

ClmDiagnosisCode_3_count = patient_data['ClmDiagnosisCode_3'].value_counts().to_dict()
patient_data['ClmDiagnosisCode_3_count']=patient_data['ClmDiagnosisCode_3'].map(ClmDiagnosisCode_3_count)

In [37]:
patient_data.shape

(558211, 92)

In [40]:
patient_data.to_csv('patient_data_file.csv')